In [ ]:
import os
import gc
import json
import math
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.layers as layers
IS_INTERACTIVE = os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'

In [ ]:
with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    char_to_num = json.load(f)

pad_token = '^'
pad_token_idx = 59

char_to_num[pad_token] = pad_token_idx

num_to_char = {j:i for i,j in char_to_num.items()}
df = pd.read_csv('/kaggle/input/asl-fingerspelling/train.csv')

LIP = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

X = [f'x_right_hand_{i}' for i in range(21)]  +[f'x_left_hand_{i}' for i in range(21)] + [f'x_pose_{i}' for i in POSE] + [f'x_face_{i}' for i in LIP] #+ 
Y = [f'y_right_hand_{i}' for i in range(21)]  +[f'y_left_hand_{i}' for i in range(21)]+ [f'y_pose_{i}' for i in POSE] + [f'y_face_{i}' for i in LIP] #+
Z = [f'z_right_hand_{i}' for i in range(21)]  + [f'z_left_hand_{i}' for i in range(21)]+ [f'z_pose_{i}' for i in POSE] + [f'z_face_{i}' for i in LIP] #+ 

SEL_COLS = X + Y + Z
FRAME_LEN = 128 + 48
MAX_PHRASE_LENGTH = 64

LIP_IDX_X   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "x" in col]
RHAND_IDX_X = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "x" in col]
LHAND_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "x" in col]
RPOSE_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "x" in col]
LPOSE_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "x" in col]

LIP_IDX_Y   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "y" in col]
RHAND_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "y" in col]
LHAND_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "y" in col]
RPOSE_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "y" in col]
LPOSE_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "y" in col]

LIP_IDX_Z   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "z" in col]
RHAND_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "z" in col]
LHAND_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "z" in col]
RPOSE_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "z" in col]
LPOSE_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "z" in col]

RHM = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/rh_mean.npy")
LHM = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/lh_mean.npy")
RPM = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/rp_mean.npy")
LPM = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/lp_mean.npy")
LIPM = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/lip_mean.npy")

RHS = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/rh_std.npy")
LHS = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/lh_std.npy")
RPS = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/rp_std.npy")
LPS = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/lp_std.npy")
LIPS = np.load("/kaggle/input/aslfr-dataset-tfrecords/mean_std/lip_std.npy")

In [ ]:
def load_relevant_data_subset(pq_path):
    return pd.read_parquet(pq_path, columns=SEL_COLS)

file_id = df.file_id.iloc[0]
inpdir = "/kaggle/input/asl-fingerspelling/train_landmarks"
pqfile = f"{inpdir}/{file_id}.parquet"
seq_refs = df.loc[df.file_id == file_id]
seqs = load_relevant_data_subset(pqfile)

seq_id = seq_refs.sequence_id.iloc[0]
frames = seqs.iloc[seqs.index == seq_id]
phrase = str(df.loc[df.sequence_id == seq_id].phrase.iloc[0])

In [ ]:
@tf.function()
def resize_pad(x):
    if tf.shape(x)[0] < FRAME_LEN:
        x = tf.pad(x, ([[0, FRAME_LEN-tf.shape(x)[0]], [0, 0], [0, 0]]), constant_values=float("NaN"))
    else:
        x = tf.image.resize(x, (FRAME_LEN, tf.shape(x)[1]))
    return x

#@tf.function()
def pre_process0(x):
    lip_x = tf.gather(x, LIP_IDX_X, axis=1)
    lip_y = tf.gather(x, LIP_IDX_Y, axis=1)
    lip_z = tf.gather(x, LIP_IDX_Z, axis=1)

    rhand_x = tf.gather(x, RHAND_IDX_X, axis=1)
    rhand_y = tf.gather(x, RHAND_IDX_Y, axis=1)
    rhand_z = tf.gather(x, RHAND_IDX_Z, axis=1)
    
    lhand_x = tf.gather(x, LHAND_IDX_X, axis=1)
    lhand_y = tf.gather(x, LHAND_IDX_Y, axis=1)
    lhand_z = tf.gather(x, LHAND_IDX_Z, axis=1)

    rpose_x = tf.gather(x, RPOSE_IDX_X, axis=1)
    rpose_y = tf.gather(x, RPOSE_IDX_Y, axis=1)
    rpose_z = tf.gather(x, RPOSE_IDX_Z, axis=1)
    
    lpose_x = tf.gather(x, LPOSE_IDX_X, axis=1)
    lpose_y = tf.gather(x, LPOSE_IDX_Y, axis=1)
    lpose_z = tf.gather(x, LPOSE_IDX_Z, axis=1)
    
    lip   = tf.concat([lip_x[..., tf.newaxis], lip_y[..., tf.newaxis], lip_z[..., tf.newaxis]], axis=-1)
    rhand = tf.concat([rhand_x[..., tf.newaxis], rhand_y[..., tf.newaxis], rhand_z[..., tf.newaxis]], axis=-1)
    lhand = tf.concat([lhand_x[..., tf.newaxis], lhand_y[..., tf.newaxis], lhand_z[..., tf.newaxis]], axis=-1)
    rpose = tf.concat([rpose_x[..., tf.newaxis], rpose_y[..., tf.newaxis], rpose_z[..., tf.newaxis]], axis=-1)
    lpose = tf.concat([lpose_x[..., tf.newaxis], lpose_y[..., tf.newaxis], lpose_z[..., tf.newaxis]], axis=-1)
        
    # TIME AUGMENTATION
    if tf.random.uniform(shape=(), minval=0, maxval=1) < 0.2:
        new_width = tf.shape(lip)[1]
        height = tf.cast(tf.shape(lip)[0], tf.float32)
        min_height = tf.cast(height / 2.0, tf.int32)
        max_height = tf.cast(height * 1.5, tf.int32)
        
        new_height = tf.random.uniform(
            shape=(), 
            minval=min_height,
            maxval=max_height,
            dtype=tf.int32
        )
        
        resized_lip = tf.image.resize(lip, (new_height, new_width))
        resized_rhand = tf.image.resize(rhand, (new_height, new_width))
        resized_lhand = tf.image.resize(lhand, (new_height, new_width))
        resized_rpose = tf.image.resize(rpose, (new_height, new_width))
        resized_lpose = tf.image.resize(lpose, (new_height, new_width))
        
        return resized_lip, resized_rhand, resized_lhand, resized_rpose, resized_lpose
        
    return lip, rhand, lhand, rpose, lpose

@tf.function(jit_compile=True)
def pre_process00(x):
    lip_x = tf.gather(x, LIP_IDX_X, axis=1)
    lip_y = tf.gather(x, LIP_IDX_Y, axis=1)
    lip_z = tf.gather(x, LIP_IDX_Z, axis=1)

    rhand_x = tf.gather(x, RHAND_IDX_X, axis=1)
    rhand_y = tf.gather(x, RHAND_IDX_Y, axis=1)
    rhand_z = tf.gather(x, RHAND_IDX_Z, axis=1)
    
    lhand_x = tf.gather(x, LHAND_IDX_X, axis=1)
    lhand_y = tf.gather(x, LHAND_IDX_Y, axis=1)
    lhand_z = tf.gather(x, LHAND_IDX_Z, axis=1)

    rpose_x = tf.gather(x, RPOSE_IDX_X, axis=1)
    rpose_y = tf.gather(x, RPOSE_IDX_Y, axis=1)
    rpose_z = tf.gather(x, RPOSE_IDX_Z, axis=1)
    
    lpose_x = tf.gather(x, LPOSE_IDX_X, axis=1)
    lpose_y = tf.gather(x, LPOSE_IDX_Y, axis=1)
    lpose_z = tf.gather(x, LPOSE_IDX_Z, axis=1)
    
    lip   = tf.concat([lip_x[..., tf.newaxis], lip_y[..., tf.newaxis], lip_z[..., tf.newaxis]], axis=-1)
    rhand = tf.concat([rhand_x[..., tf.newaxis], rhand_y[..., tf.newaxis], rhand_z[..., tf.newaxis]], axis=-1)
    lhand = tf.concat([lhand_x[..., tf.newaxis], lhand_y[..., tf.newaxis], lhand_z[..., tf.newaxis]], axis=-1)
    rpose = tf.concat([rpose_x[..., tf.newaxis], rpose_y[..., tf.newaxis], rpose_z[..., tf.newaxis]], axis=-1)
    lpose = tf.concat([lpose_x[..., tf.newaxis], lpose_y[..., tf.newaxis], lpose_z[..., tf.newaxis]], axis=-1)
                
    hand = tf.concat([rhand, lhand], axis=1)
    hand = tf.where(tf.math.is_nan(hand), 0.0, hand)
    input_tensor = tf.math.not_equal(tf.reduce_sum(hand, axis=[1, 2]), 0.0)
    alternating_tensor = tf.math.equal( tf.cumsum(tf.ones_like( tf.reduce_sum(hand, axis=[1, 2]) ))%2, 1.0 )
    mask = tf.math.logical_or(input_tensor, alternating_tensor)
    
    lip = lip[mask]
    rhand = rhand[mask]
    lhand = lhand[mask]
    rpose = rpose[mask]
    lpose = lpose[mask]

    return lip, rhand, lhand,  rpose, lpose

@tf.function()
def pre_process1(lip, rhand,lhand,  rpose, lpose): #lhand,
    lip   = (resize_pad(lip) - LIPM) / LIPS
    rhand = (resize_pad(rhand) - RHM) / RHS
    lhand = (resize_pad(lhand) - LHM) / LHS
    rpose = (resize_pad(rpose) - RPM) / RPS
    lpose = (resize_pad(lpose) - LPM) / LPS

    x = tf.concat([lip, rhand, lhand, rpose, lpose], axis=1) #lhand,
    s = tf.shape(x)
    x = tf.reshape(x, (s[0], s[1]*s[2]))
    x = tf.where(tf.math.is_nan(x), 0.0, x)
    return x

pre0 = pre_process0(frames)
pre1 = pre_process1(*pre0)
INPUT_SHAPE = list(pre1.shape)
print(INPUT_SHAPE)

In [ ]:
def decode_fn(record_bytes):
    schema = {
        "lip": tf.io.VarLenFeature(tf.float32),
        "rhand": tf.io.VarLenFeature(tf.float32),
        "lhand": tf.io.VarLenFeature(tf.float32),
        "rpose": tf.io.VarLenFeature(tf.float32),
        "lpose": tf.io.VarLenFeature(tf.float32),
        "phrase": tf.io.VarLenFeature(tf.int64)
    }
    x = tf.io.parse_single_example(record_bytes, schema)

    lip = tf.reshape(tf.sparse.to_dense(x["lip"]), (-1, 40, 3))
    rhand = tf.reshape(tf.sparse.to_dense(x["rhand"]), (-1, 21, 3))
    lhand = tf.reshape(tf.sparse.to_dense(x["lhand"]), (-1, 21, 3))
    rpose = tf.reshape(tf.sparse.to_dense(x["rpose"]), (-1, 5, 3))
    lpose = tf.reshape(tf.sparse.to_dense(x["lpose"]), (-1, 5, 3))
    phrase = tf.sparse.to_dense(x["phrase"])

    return lip, rhand, lhand,  rpose, lpose, phrase #lhand,

def pre_process_fn(lip, rhand,lhand,  rpose, lpose, phrase): #lhand,
    phrase = tf.pad(phrase, [[0, MAX_PHRASE_LENGTH-tf.shape(phrase)[0]]], constant_values=pad_token_idx)
    return pre_process1(lip, rhand, lhand, rpose, lpose), phrase #lhand,
    
tffiles = [f"/kaggle/input/chris-tf-v9/{file_id}.tfrecord" for file_id in df.file_id.unique()]
val_len = 1
train_batch_size = 64
val_batch_size = 64

train_dataset = tf.data.TFRecordDataset(tffiles[val_len:]).prefetch(tf.data.AUTOTUNE).shuffle(5000)\
    .map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE)\
    .map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE)\
    .batch(train_batch_size)\
    .prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.TFRecordDataset(tffiles[:val_len]).prefetch(tf.data.AUTOTUNE)\
    .map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE)\
    .map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE)\
    .batch(val_batch_size)\
    .prefetch(tf.data.AUTOTUNE)

batch = next(iter(val_dataset))
batch[0].shape, batch[1].shape

In [ ]:
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self, 
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True
        
    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x

class SqueezeExcite(tf.keras.layers.Layer):
    def __init__(self, channels, reduction_ratio=8, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.gap = tf.keras.layers.GlobalAveragePooling1D()
        reduced_channels = max(1, channels // reduction_ratio)
        self.fc1 = tf.keras.layers.Dense(reduced_channels, activation='swish')
        self.fc2 = tf.keras.layers.Dense(channels, activation='sigmoid')
        
    def call(self, inputs, mask=None):
        x = self.gap(inputs, mask=mask)
        x = self.fc1(x)
        x = self.fc2(x)
        return inputs * tf.expand_dims(x, axis=1)

class ConvModule(tf.keras.layers.Layer):
    def __init__(self, dim, kernel_size, expansion_factor=2, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.conv1 = tf.keras.layers.Conv1D(dim * expansion_factor, 1)
        self.conv2 = CausalDWConv1D(kernel_size=kernel_size)
        self.se = SqueezeExcite(dim)
        self.final_conv = tf.keras.layers.Conv1D(dim, 1)  # Ensure output dimension matches input
        
    def call(self, inputs, mask=None):
        x = self.norm(inputs)
        x = self.conv1(x)
        x = tf.keras.activations.swish(x)
        x = self.conv2(x)
        x = tf.keras.activations.swish(x)
        x = self.final_conv(x)  # Add this to reduce dimensionality if needed
        x = self.se(x, mask=mask)
        return x + inputs

class SqueezeformerBlock(tf.keras.layers.Layer):
    def __init__(self, dim, num_heads=8, expansion_factor=4, kernel_size=31, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        
        # Feed Forward Module 1
        self.norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ffn1 = tf.keras.Sequential([
            tf.keras.layers.Dense(dim * expansion_factor, activation='swish'),
            tf.keras.layers.Dropout(dropout),
            tf.keras.layers.Dense(dim)
        ])
        
        # Multi-head Self Attention
        self.norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.mha = MultiHeadSelfAttention(dim=dim, num_heads=num_heads, dropout=dropout)
        
        # Convolution Module
        self.conv = ConvModule(dim, kernel_size, expansion_factor)
        
        # Feed Forward Module 2
        self.norm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ffn2 = tf.keras.Sequential([
            tf.keras.layers.Dense(dim * expansion_factor, activation='swish'),
            tf.keras.layers.Dropout(dropout),
            tf.keras.layers.Dense(dim)
        ])
        
        self.dropout = tf.keras.layers.Dropout(dropout)
        
    def call(self, inputs, mask=None):
        # First FFN
        residual = inputs
        x = self.norm1(inputs)
        x = self.ffn1(x)
        x = residual + self.dropout(x)
        
        # Self Attention
        residual = x
        x = self.norm2(x)
        x = self.mha(x, mask=mask)
        x = residual + self.dropout(x)
        
        # Convolution
        x = self.conv(x, mask=mask)
        
        # Second FFN
        residual = x
        x = self.norm3(x)
        x = self.ffn2(x)
        x = residual + self.dropout(x)
        
        return x
        
def TransformerBlock(dim=256, num_heads=6, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply

class ConvolutionModule(layers.Layer):
    def __init__(self, dim, kernel_size=31, dropout=0.1, name=None):
        super(ConvolutionModule, self).__init__(name=name)
        self.dim = dim
        self.kernel_size = kernel_size
        self.dropout = dropout

    def build(self, input_shape):
        # Pointwise Convolution 1
        self.pointwise_conv1 = layers.Conv1D(
            filters=self.dim * 2, 
            kernel_size=1, 
            name='pointwise_conv1'
        )
        
        # Depthwise Convolution
        self.depthwise_conv = layers.Conv1D(
            filters=self.dim, 
            kernel_size=self.kernel_size, 
            groups=self.dim, 
            padding='same', 
            name='depthwise_conv'
        )
        
        # Pointwise Convolution 2
        self.pointwise_conv2 = layers.Conv1D(
            filters=self.dim, 
            kernel_size=1, 
            name='pointwise_conv2'
        )
        
        # Batch Normalization
        self.batch_norm = layers.BatchNormalization(name='batch_norm')
        
        # Dropout
        self.dropout_layer = layers.Dropout(self.dropout, name='dropout')
        
        # Layer Normalization
        self.layer_norm = layers.LayerNormalization(name='layer_norm')
        
        super().build(input_shape)

    def call(self, x, training=None):
        # Store residual connection
        residual = x
        
        # Transpose for convolution operations
        x = tf.transpose(x, perm=[0, 2, 1])
        
        # Pointwise Conv 1 + GLU
        x = self.pointwise_conv1(x)
        x = tf.split(x, num_or_size_splits=2, axis=-1)
        x = x[0] * tf.sigmoid(x[1])  # Manually implement GLU
        
        # Depthwise Convolution
        x = self.depthwise_conv(x)
        
        # Batch Normalization
        x = self.batch_norm(x, training=training)
        
        # Pointwise Conv 2
        x = self.pointwise_conv2(x)
        
        # Transpose back
        x = tf.transpose(x, perm=[0, 2, 1])
        
        # Residual connection and Layer Normalization
        x = self.layer_norm(x + residual)
        
        return x
        
def positional_encoding(maxlen, num_hid):
        depth = num_hid/2
        positions = tf.range(maxlen, dtype = tf.float32)[..., tf.newaxis]
        depths = tf.range(depth, dtype = tf.float32)[np.newaxis, :]/depth
        angle_rates = tf.math.divide(1, tf.math.pow(tf.cast(10000, tf.float32), depths))
        angle_rads = tf.linalg.matmul(positions, angle_rates)
        pos_encoding = tf.concat(
          [tf.math.sin(angle_rads), tf.math.cos(angle_rads)],
          axis=-1)
        return pos_encoding

class FeedForwardModule(tf.keras.layers.Layer):
    def __init__(self, dim, expansion_factor=4, dropout=0.1, name=None):
        super().__init__(name=name)
        self.sequential = tf.keras.Sequential([
            tf.keras.layers.Dense(dim * expansion_factor, activation='swish'),
            tf.keras.layers.Dropout(dropout),
            tf.keras.layers.Dense(dim)
        ])
    
    def call(self, x, training=None):
        return self.sequential(x, training=training)

class ConvolutionModule(tf.keras.layers.Layer):
    def __init__(self, dim, kernel_size=31, dropout=0.1, name=None):
        super(ConvolutionModule, self).__init__(name=name)
        self.dim = dim
        self.kernel_size = kernel_size
        self.dropout = dropout

    def build(self, input_shape):
        # Pointwise Convolution 1
        self.pointwise_conv1 = tf.keras.layers.Conv1D(
            filters=self.dim * 2, 
            kernel_size=1, 
            name='pointwise_conv1'
        )
        
        # Depthwise Convolution
        self.depthwise_conv = tf.keras.layers.Conv1D(
            filters=input_shape[-1], 
            kernel_size=self.kernel_size, 
            padding='same', 
            groups=input_shape[-1],
            name='depthwise_conv'
        )
        
        # Pointwise Convolution 2
        self.pointwise_conv2 = tf.keras.layers.Conv1D(
            filters=input_shape[-1], 
            kernel_size=1, 
            name='pointwise_conv2'
        )
        
        # Batch Normalization
        self.batch_norm = tf.keras.layers.BatchNormalization(name='batch_norm')
        
        # Layer Normalization
        self.layer_norm = tf.keras.layers.LayerNormalization(name='layer_norm')
        
        super().build(input_shape)

    def call(self, x, training=None):
        # Store residual connection
        residual = x
        
        # Pointwise Conv 1 + GLU
        x = self.pointwise_conv1(x)
        x = tf.split(x, num_or_size_splits=2, axis=-1)
        x = x[0] * tf.sigmoid(x[1])  # Manually implement GLU
        
        # Depthwise Convolution
        x = self.depthwise_conv(x)
        
        # Batch Normalization
        x = self.batch_norm(x, training=training)
        
        # Pointwise Conv 2
        x = self.pointwise_conv2(x)
        
        # Residual connection and Layer Normalization
        x = self.layer_norm(x + residual)
        
        return x

class ConformerBlock(tf.keras.layers.Layer):
    def __init__(self, dim, num_heads=8, expand=4, kernel_size=31, attn_dropout=0.1, drop_rate=0.1, activation='swish', name=None):
        super().__init__(name=name)
        self.ffn1 = FeedForwardModule(dim, expand, drop_rate, name='ffn1')
        self.mha = MultiHeadSelfAttention(dim=dim, num_heads=num_heads, dropout=attn_dropout)
        self.conv = ConvolutionModule(dim, kernel_size, drop_rate)
        self.ffn2 = FeedForwardModule(dim, expand, drop_rate, name='ffn2')
        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training=None, mask=None):
        # First Feed Forward Module (1/2)
        residual = x
        x = self.layer_norm1(x)
        x = self.ffn1(x, training=training)
        x = residual + x
        
        # Multi-Head Self-Attention Module
        residual = x
        x = self.layer_norm1(x)
        x = self.mha(x, mask=mask)
        x = residual + x
        
        # Convolution Module
        x = self.conv(x, training=training)
        
        # Second Feed Forward Module (2/2)
        residual = x
        x = self.layer_norm2(x)
        x = self.ffn2(x, training=training)
        x = residual + x
        
        return x

In [ ]:
def CTCLoss(labels, logits):
    label_length = tf.reduce_sum(tf.cast(labels != pad_token_idx, tf.int32), axis=-1)
    logit_length = tf.ones(tf.shape(logits)[0], dtype=tf.int32) * tf.shape(logits)[1]
    loss = tf.nn.ctc_loss(
            labels=labels,
            logits=logits,
            label_length=label_length,
            logit_length=logit_length,
            blank_index=pad_token_idx,
            logits_time_major=False
        )
    loss = tf.reduce_mean(loss)
    return loss

In [ ]:
def get_model(dim=276):
    inp = tf.keras.Input(INPUT_SHAPE)
    x = tf.keras.layers.Masking(mask_value=0.0)(inp)
    
    # Stem convolution with positional encoding
    x = tf.keras.layers.Dense(dim, use_bias=False, name='stem_conv')(x)
    pe = tf.cast(positional_encoding(INPUT_SHAPE[0], dim), dtype=x.dtype)
    x = x + pe
    x = tf.keras.layers.BatchNormalization(momentum=0.95, name='stem_bn')(x)
    
    # Parallel Conformer and Squeezeformer blocks
    conformer_branch = x
    squeezeformer_branch = x
    
    num_blocks = 2
    drop_rate = 0.4
    
    # Conformer blocks
    for i in range(num_blocks):
        conformer_branch = ConformerBlock(
            dim=dim,
            num_heads=4,
            expand=2,
            attn_dropout=drop_rate,
            drop_rate=drop_rate,
            activation='swish',
            kernel_size=15
        )(conformer_branch)
    
    # Squeezeformer blocks
    for i in range(num_blocks):
        squeezeformer_branch = SqueezeformerBlock(
            dim=dim,
            num_heads=4,
            expansion_factor=2,
            kernel_size=15,
            dropout=drop_rate
        )(squeezeformer_branch)
    
    # Merge the branches
    x = tf.keras.layers.Concatenate()([conformer_branch, squeezeformer_branch])
    
    # Reduce dimensionality after concatenation
    x = tf.keras.layers.Dense(dim, activation='relu', name='merge_conv')(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    
    # Output layer - time-distributed classification
    x = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(len(char_to_num), activation='softmax', name='classifier')
    )(x)
    
    model = tf.keras.Model(inp, x)
    
    # Use categorical crossentropy for time-distributed output
    loss = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tfa.optimizers.RectifiedAdam(sma_threshold=4)
    optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=5)
    
    model.compile(loss=loss, optimizer=optimizer)
    
    return model

tf.keras.backend.clear_session()
model = get_model()
model(batch[0])
model.summary()

In [ ]:
def num_to_char_fn(y):
    return [num_to_char.get(x, "") for x in y]

@tf.function()
def decode_phrase(pred):
    x = tf.argmax(pred, axis=1)
    diff = tf.not_equal(x[:-1], x[1:])
    adjacent_indices = tf.where(diff)[:, 0]
    x = tf.gather(x, adjacent_indices)
    mask = x != pad_token_idx
    x = tf.boolean_mask(x, mask, axis=0)
    return x

# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    output_text = []
    for result in pred:
        result = "".join(num_to_char_fn(decode_phrase(result).numpy()))
        output_text.append(result)
    return output_text

In [ ]:

# A callback class to output a few transcriptions during training
class CallbackEval(tf.keras.callbacks.Callback):
    """Displays a batch of outputs after every epoch."""

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch: int, logs=None):
        model.save_weights("model.h5")
        predictions = []
        targets = []
        for batch in self.dataset:
            X, y = batch
            batch_predictions = model(X)
            batch_predictions = decode_batch_predictions(batch_predictions)
            predictions.extend(batch_predictions)
            for label in y:
                label = "".join(num_to_char_fn(label.numpy()))
                targets.append(label)
        print("-" * 100)
        # for i in np.random.randint(0, len(predictions), 2):
        for i in range(32):
            print(f"Target    : {targets[i]}")
            print(f"Prediction: {predictions[i]}, len: {len(predictions[i])}")
            print("-" * 100)

# Callback function to check transcription on the val set.
validation_callback = CallbackEval(val_dataset.take(1))

In [ ]:
N_EPOCHS = 2 if IS_INTERACTIVE else 50
N_WARMUP_EPOCHS = 0 if IS_INTERACTIVE else 5
LR_MAX = 4e-3
WD_RATIO = 0.05
WARMUP_METHOD = "exp"

In [ ]:
def lrfn(current_step, num_warmup_steps, lr_max, num_cycles=0.50, num_training_steps=N_EPOCHS):
    
    if current_step < num_warmup_steps:
        if WARMUP_METHOD == 'log':
            return lr_max * 0.10 ** (num_warmup_steps - current_step)
        else:
            return lr_max * 2 ** -(num_warmup_steps - current_step)
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))) * lr_max
    
def plot_lr_schedule(lr_schedule, epochs):
    fig = plt.figure(figsize=(20, 10))
    plt.plot([None] + lr_schedule + [None])
    # X Labels
    x = np.arange(1, epochs + 1)
    x_axis_labels = [i if epochs <= 40 or i % 5 == 0 or i == 1 else None for i in range(1, epochs + 1)]
    plt.xlim([1, epochs])
    plt.xticks(x, x_axis_labels) # set tick step to 1 and let x axis start at 1
    
    # Increase y-limit for better readability
    plt.ylim([0, max(lr_schedule) * 1.1])
    
    # Title
    schedule_info = f'start: {lr_schedule[0]:.1E}, max: {max(lr_schedule):.1E}, final: {lr_schedule[-1]:.1E}'
    plt.title(f'Step Learning Rate Schedule, {schedule_info}', size=18, pad=12)
    
    # Plot Learning Rates
    for x, val in enumerate(lr_schedule):
        if epochs <= 40 or x % 5 == 0 or x is epochs - 1:
            if x < len(lr_schedule) - 1:
                if lr_schedule[x - 1] < val:
                    ha = 'right'
                else:
                    ha = 'left'
            elif x == 0:
                ha = 'right'
            else:
                ha = 'left'
            plt.plot(x + 1, val, 'o', color='black');
            offset_y = (max(lr_schedule) - min(lr_schedule)) * 0.02
            plt.annotate(f'{val:.1E}', xy=(x + 1, val + offset_y), size=12, ha=ha)
    
    plt.xlabel('Epoch', size=16, labelpad=5)
    plt.ylabel('Learning Rate', size=16, labelpad=5)
    plt.grid()
    plt.show()

# Learning rate for encoder
LR_SCHEDULE = [lrfn(step, num_warmup_steps=N_WARMUP_EPOCHS, lr_max=LR_MAX, num_cycles=0.50) for step in range(N_EPOCHS)]
# Plot Learning Rate Schedule
plot_lr_schedule(LR_SCHEDULE, epochs=N_EPOCHS)
# Learning Rate Callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step], verbose=0)

# Custom callback to update weight decay with learning rate
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, wd_ratio=WD_RATIO):
        self.step_counter = 0
        self.wd_ratio = wd_ratio
    
    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.learning_rate * self.wd_ratio
        print(f'learning rate: {model.optimizer.learning_rate.numpy():.2e}, weight decay: {model.optimizer.weight_decay.numpy():.2e}')

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=N_EPOCHS,
    callbacks=[
        validation_callback,
        lr_callback,
        WeightDecayCallback(),
    ]
)

In [ ]:
class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, len(SEL_COLS)], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs, training=False):
        # Preprocess Data
        x = tf.cast(inputs, tf.float32)
        x = x[None]
        x = tf.cond(tf.shape(x)[1] == 0, lambda: tf.zeros((1, 1, len(SEL_COLS))), lambda: tf.identity(x))
        x = x[0]
        x = pre_process00(x)
        x = pre_process1(*x)
        x = tf.reshape(x, INPUT_SHAPE)
        x = x[None]
        x = self.model(x, training=False)
        x = x[0]
        x = decode_phrase(x)
        # POST PROCESS. IF PRED LESS THAN 3, USE CONSTANT PREDICTION FROM
        # https://www.kaggle.com/code/anokas/static-greedy-baseline-0-157-lb
        x = tf.cond(tf.shape(x)[0] < 3, lambda: tf.constant(
            [17, 0, 32, 12, 36, 0, 12, 32, 49, 46, 36], tf.int64), lambda: tf.identity(x))
        x = tf.one_hot(x, 59)
        return {'outputs': x}

tflitemodel_base = TFLiteModel(model)
tflitemodel_base(frames)["outputs"].shape

In [ ]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]#, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]
keras_model_converter.target_spec.supported_types = [tf.float16]
tflite_model = keras_model_converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
    
with open('inference_args.json', "w") as f:
    json.dump({"selected_columns" : SEL_COLS}, f)
    
!zip submission.zip  './model.tflite' './inference_args.json'

In [ ]:
with open ("inference_args.json", "r") as f:
    SEL_COLS = json.load(f)["selected_columns"]
    
def load_relevant_data_subset(pq_path):
    return pd.read_parquet(pq_path, columns=SEL_COLS)

def create_data_gen(file_ids, y_mul=1):
    def gen():
        for file_id in file_ids:
            pqfile = f"{inpdir}/{file_id}.parquet"
            seq_refs = df.loc[df.file_id == file_id]
            seqs = load_relevant_data_subset(pqfile)

            for seq_id in seq_refs.sequence_id:
                x = seqs.iloc[seqs.index == seq_id].to_numpy()
                y = str(df.loc[df.sequence_id == seq_id].phrase.iloc[0])
                
                r_nonan = np.sum(np.sum(np.isnan(x[:, RHAND_IDX_X]), axis = 1) == 0)
                l_nonan = np.sum(np.sum(np.isnan(x[:, LHAND_IDX_X]), axis = 1) == 0)
                no_nan = max(r_nonan, l_nonan)
                
                if y_mul*len(y)<no_nan:
                    yield x, y
    return gen

pqfiles = df.file_id.unique()
val_len = int(0.05 * len(pqfiles))

test_dataset = tf.data.Dataset.from_generator(create_data_gen(pqfiles[:val_len], 0),
    output_signature=(tf.TensorSpec(shape=(None, len(SEL_COLS)), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.string))
).prefetch(buffer_size=2000)

In [ ]:
interpreter = tf.lite.Interpreter("model.tflite")

REQUIRED_SIGNATURE = "serving_default"
REQUIRED_OUTPUT = "outputs"

with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)
rev_character_map = {j:i for i,j in character_map.items()}

prediction_fn = interpreter.get_signature_runner(REQUIRED_SIGNATURE)

for frame, target in test_dataset.skip(100).take(10):
    output = prediction_fn(inputs=frame)
    prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)])
    target = target.numpy().decode("utf-8")
    print("pred =", prediction_str, "; target =", target)

In [ ]:
%%timeit -n 10
output = prediction_fn(inputs=frame)

In [ ]:
from Levenshtein import distance

scores = []

for i, (frame, target) in tqdm(enumerate(test_dataset.take(1000))):
    output = prediction_fn(inputs=frame)
    prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)])
    target = target.numpy().decode("utf-8")
    score = (len(target) - distance(prediction_str, target)) / len(target)
    scores.append(score)
    if i % 50 == 0:
        print(np.sum(scores) / len(scores))
    
scores = np.array(scores)
print(np.sum(scores) / len(scores))